In [6]:
import sys
from experanto.datasets import ChunkDataset
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import OrderedDict

In [12]:
root_folder = '../data/example_experiment'
sampling_rate = 30  # Timestamps generated by this do not match the real ones, why do this? Is this for output data?
chunk_size = 20

In [13]:
train_dataset = ChunkDataset(root_folder=root_folder, global_sampling_rate=sampling_rate,
            global_chunk_size=chunk_size,
            modality_config = 
            {'screen': {
                'sampling_rate': None,
                'chunk_size': None,
                'valid_condition': {
                    'tier': 'train',
                    'stim_type': 'stimulus.Frame', #include both images and videos
                    'stim_type': 'stimulus.Clip'
                },
                'offset': 0,
                'sample_stride': 4,
                # necessary for the allen dataset since there are blanks after every stimuli because else no valid times are found
                'include_blanks': True, 
                'transforms': {
                    'ToTensor': {
                        '_target_': 'torchvision.transforms.ToTensor'
                    },
                    'Normalize': {
                        '_target_': 'torchvision.transforms.Normalize',
                        'mean': 80.0,
                        'std': 60.0
                    },
                    'Resize': {
                        '_target_': 'torchvision.transforms.Resize',
                        'size': [144, 256]
                    },
                    'CenterCrop': {
                        '_target_': 'torchvision.transforms.CenterCrop',
                        'size': 144
                    }
                },
                'interpolation': {}
            },
            'responses': {
                'sampling_rate': None,
                'chunk_size': None,
                'offset': 0.1,
                'transforms': {
                    'standardize': True
                },
                'interpolation': {
                    'interpolation_mode': 'nearest_neighbor'
                }
            },
            'eye_tracker': {
                'sampling_rate': None,
                'chunk_size': None,
                'offset': 0,
                'transforms': {
                    'normalize': True
                },
                'interpolation': {
                    'interpolation_mode': 'nearest_neighbor'
                }
            },
            'treadmill': {
                'sampling_rate': None,
                'chunk_size': None,
                'offset': 0,
                'transforms': {
                    'normalize': True
                },
                'interpolation': {
                    'interpolation_mode': 'nearest_neighbor'
                }
            }
        })

/usr/local/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [15]:
batch_size = 1
data_loaders = OrderedDict()
m = 'example_experiment'

data_loaders['train'] = OrderedDict()
#data_loaders['oracle'] = OrderedDict()
data_loaders['train'][m] =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#data_loaders['oracle'][m] = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
it =next(iter(data_loaders['train'][m]))

In [24]:
for key in it.keys():
    print(f"This is shape of {key} : {it[key].shape}")

This is shape of eye_tracker : torch.Size([1, 20, 22])
This is shape of screen : torch.Size([1, 20, 1, 144, 144])
This is shape of treadmill : torch.Size([1, 20, 1])
This is shape of responses : torch.Size([1, 20, 12])
This is shape of timestamps : torch.Size([1, 20])
